从Google Driver中下载并导入数据

In [17]:
import sys
import pandas as pd
import numpy as np
from google.colab import drive 
!gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip
!unzip data.zip
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
data = pd.read_csv('./train.csv', encoding = 'big5')

Downloading...
From: https://drive.google.com/uc?id=1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm
To: /content/data.zip
100% 177k/177k [00:00<00:00, 86.1MB/s]
Archive:  data.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [18]:
print(data)

              日期  測站          測項     0     1  ...    19    20    21    22    23
0       2014/1/1  豐原    AMB_TEMP    14    14  ...    16    15    15    15    15
1       2014/1/1  豐原         CH4   1.8   1.8  ...   1.8   1.8   1.8   1.8   1.8
2       2014/1/1  豐原          CO  0.51  0.41  ...  0.45  0.38  0.35  0.36  0.32
3       2014/1/1  豐原        NMHC   0.2  0.15  ...  0.12   0.1  0.09   0.1  0.08
4       2014/1/1  豐原          NO   0.9   0.6  ...   1.9   1.5   1.6   1.8   1.5
...          ...  ..         ...   ...   ...  ...   ...   ...   ...   ...   ...
4315  2014/12/20  豐原         THC   1.8   1.8  ...   1.9   1.9   1.9     2     2
4316  2014/12/20  豐原       WD_HR    46    13  ...   109   108   114   108   109
4317  2014/12/20  豐原  WIND_DIREC    36    55  ...    97   107   118   100   105
4318  2014/12/20  豐原  WIND_SPEED   1.9   2.4  ...     1   1.7   1.5     2     2
4319  2014/12/20  豐原       WS_HR   0.7   0.8  ...   1.4   1.3   1.6   1.8     2

[4320 rows x 27 columns]


提取出数据部分

In [19]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
print(raw_data)

[['14' '14' '14' ... '15' '15' '15']
 ['1.8' '1.8' '1.8' ... '1.8' '1.8' '1.8']
 ['0.51' '0.41' '0.39' ... '0.35' '0.36' '0.32']
 ...
 ['36' '55' '72' ... '118' '100' '105']
 ['1.9' '2.4' '1.9' ... '1.5' '2' '2']
 ['0.7' '0.8' '1.8' ... '1.6' '1.8' '2']]


按月份将数据分块，每18行为一天，一共4320/18=240天</br>
数据中每月记录有20天</br>
每一天中的数据格式为 18个特征 * 24小时</br>
先将每月每一天的18 * 24大小的数据单独提取出存入sample数组中</br>
sample数组的大小是18 * 480，其中480是每天的24小时 * 每月的20天计算出

In [0]:
month_data = {}
for month in range(12):
  sample = np.empty([18, 480])
  for day in range(20):
    sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
  month_data[month] = sample

题目要求使用每天前9小时的数据进行训练</br>
每个月一共有480小时，每9小时组成一项数据，每月有471项数据</br>
所以数据总数为471 * 12个月份</br>
每项数据中内容是9个小时 * 18个特征</br>
对应的标签为每第10个小时的PM2.5的值，一共有471 * 12个

In [21]:
x = np.empty([12 * 471, 18 * 9], dtype = float)#训练数据
y = np.empty([12 * 471, 1], dtype = float)#标签数据
for month in range(12):
  for day in range(20):
    for hour in range(24):
      if day == 19 and hour > 14:
        continue
      x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)
      y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9]
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


训练数据归一化

In [22]:
mean_x = np.mean(x, axis = 0)#平均值
std_x = np.std(x, axis = 0)#标准差
for i in range(len(x)):
  for j in range(len(x[0])):
    if std_x[j] != 0:
      x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
print(x)

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.38950555 -0.10906991
   0.07797893]]


将训练集拆分成训练集和验证集，比例约为4:1

In [23]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]#向下取整
y_train_set = y[: math.floor(len(x) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(x) * 0.8): , :]
print(len(x_train_set))
print(len(x_train_set[0]))

4521
162


训练</br>
使用线性模型，将9小时的18个特征都代入到模型中，均为一次，再加上一个偏置项

In [24]:
dim = 18 * 9 + 1  #一项数据的长度为18 * 9，再加上一个常数项
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([math.floor(12 * 471 * 0.8), 1]), x_train_set), axis = 1).astype(float)
learning_rate = 100
iter_times = 1000
adagrad = np.zeros([dim, 1]) #用于记录累积的梯度平方
eps = 0.0000000001  #用于防止梯度为0
for t in range(iter_times):
  loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y_train_set, 2)) / (math.floor(12 * 471 * 0.8))) #RMSE，loss为平均每项数据计算出的预测值与真实值的差值的平方，再取根号
  if (t % 100 == 0 or t == iter_times - 1):
    print(str(t) + ":" + str(loss))
  gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y_train_set)
  adagrad += np.power(gradient, 2)
  w = w - learning_rate * gradient / np.sqrt(adagrad + eps)

0:27.239591682144457
100:22.723975744599546
200:10.427772652266519
300:7.635824772204668
400:6.849878685598302
500:6.518567463859378
600:6.32564308053946
700:6.1943518687643575
800:6.09908116190811
900:6.027838242824769
999:5.974024879930956


最终在训练集上的表现

In [25]:
x = np.concatenate((np.ones([math.floor(12 * 471 * 0.8), 1]), x_train_set), axis = 1).astype(float)
loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y_train_set, 2)) / (math.floor(12 * 471 * 0.8)))
loss

5.973549167998369

在验证集上的表现

In [27]:
x = np.concatenate((np.ones([12 * 471 - math.floor(12 * 471 * 0.8), 1]), x_validation), axis = 1).astype(float)
loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y_validation, 2)) / (12 * 471 - math.floor(12 * 471 * 0.8)))
loss

5.91220546628651

尝试如果只使用前9天18个天气特征中的PM2.5特征进行训练</br>
数据重新导入

In [34]:
x = np.empty([12 * 471, 18 * 9], dtype = float)#训练数据
y = np.empty([12 * 471, 1], dtype = float)#标签数据
for month in range(12):
  for day in range(20):
    for hour in range(24):
      if day == 19 and hour > 14:
        continue
      x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)
      y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9]
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


PM2.5数据在每小时数据的第10个

In [43]:
x_pm25 = x[: , 9 * 9 : 9 * 9 + 9]
y_pm25 = y
print(x_pm25)
print(y_pm25)

[[26. 39. 36. ... 25. 20. 19.]
 [39. 36. 35. ... 20. 19. 30.]
 [36. 35. 31. ... 19. 30. 41.]
 ...
 [ 4.  1.  1. ... 49. 37. 28.]
 [ 1.  1.  7. ... 37. 28. 17.]
 [ 1.  7. 24. ... 28. 17. 24.]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


将训练数据归一化并划分出验证集

In [49]:
x_pm25_mean = np.mean(x_pm25, axis = 0)
x_pm25_std = np.std(x_pm25, axis = 0)
for i in range(len(x_pm25)):
  for j in range(len(x_pm25[0])):
    if x_pm25_std[j] != 0:
      x_pm25[i][j] = (x_pm25[i][j] - x_pm25_mean[j]) / x_pm25_std[j]
x_pm25_train = x_pm25[: math.floor(len(x_pm25) * 0.8), : ]
y_pm25_train = y_pm25[: math.floor(len(x_pm25) * 0.8), : ]
x_pm25_valid = x_pm25[math.floor(len(x_pm25) * 0.8) : , : ]
y_pm25_valid = y_pm25[math.floor(len(x_pm25) * 0.8) : , : ]
print(len(x_pm25_train))
print(x_pm25)

4521
[[ 0.27889755  1.05839335  0.88014061 ...  0.21838973 -0.08226916
  -0.14272026]
 [ 1.05656354  0.87873275  0.82016285 ... -0.08214074 -0.1424225
   0.51926154]
 [ 0.87710216  0.81884589  0.58025185 ... -0.14224684  0.51926421
   1.18124334]
 ...
 [-1.03715259 -1.21730758 -1.21908069 ...  1.660936    0.94033757
   0.39890121]
 [-1.21661397 -1.21730758 -0.85921419 ...  0.93966286  0.39895754
  -0.26308059]
 [-1.21661397 -0.85798638  0.16040759 ...  0.39870801 -0.26272917
   0.15818056]]


训练</br>
使用线性模型，将9小时的PM2.5都代入到模型中，均为一次，再加上一个偏置项

In [51]:
dim_pm25 = 9 + 1
w_pm25 = np.zeros([dim_pm25, 1])
x = np.concatenate((np.ones([len(x_pm25_train), 1]), x_pm25_train), axis = 1).astype(float)
learning_rate = 100
iter_times = 1000
adagrad = np.zeros([dim_pm25, 1])
eps = 0.0000000001
for t in range(iter_times):
  loss = np.sqrt(np.sum(np.power(np.dot(x, w_pm25) - y_pm25_train, 2)) / len(x_pm25_train))
  if t % 100 == 0 or t == iter_times - 1:
    print(str(t) + ":" + str(loss))
  gradient = 2 * np.dot(x.transpose(), np.dot(x, w_pm25) - y_pm25_train)
  adagrad += np.power(gradient, 2)
  w_pm25 = w_pm25 - learning_rate * gradient / np.sqrt(adagrad + eps)

0:27.239591682144457
100:6.475116994173631
200:6.299428696470295
300:6.239356702474483
400:6.214218284941439
500:6.203148310110213
600:6.198180209744697
700:6.195929908267414
800:6.194904606778375
900:6.194435146005474
999:6.194220577768057


最终在训练集上的表现为

In [52]:
x = np.concatenate((np.ones([len(x_pm25_train), 1]), x_pm25_train), axis = 1).astype(float)
loss = np.sqrt(np.sum(np.power(np.dot(x, w_pm25) - y_pm25_train, 2)) / len(x_pm25_train))
loss

6.194219140254146

在验证集上的表现为

In [53]:
x = np.concatenate((np.ones([len(x_pm25_valid), 1]), x_pm25_valid), axis = 1).astype(float)
loss = np.sqrt(np.sum(np.power(np.dot(x, w_pm25) - y_pm25_valid, 2)) / len(x_pm25_valid))
loss

5.86461175212293

# **总结**

---

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;线性回归主要是用线性函数去拟合数据之间的关系，从而对不在数据集中的数据进行预测。</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;本次作业中数据集提供的天气特征有18个，用前9小时的总计9 * 18个数据对第10小时的其中一个特征PM2.5进行预测。</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;使用哪些天气特征代入线性回归是一个要讨论的问题，作业中首先尝试了使用所有的9 * 18个数据代入线性回归，最后在训练集合验证集上的误差，与只使用9个PM2.5数据进行预测的误差大致相同。</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;所以在天气特征中有一部分是对PM2.5值预测关系不大的，在制定线性模型时应该综合考量每个特征之间的关系，尽可能减少模型中不需要的特征变量。